In [16]:
import pandas as pd
import numpy as np 
from sklearn.cluster import AgglomerativeClustering

In [4]:
data = pd.read_csv("data/ag_news/ag_news_train.csv").sample(100)

In [5]:
data.shape

(100, 3)

In [6]:
!pip install bert-embedding==1.0.1

     |████████████████████████████████| 29.6 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 209 kB 25.7 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 31.9 MB/s eta 0:00:01
  Created wheel for gluonnlp: filename=gluonnlp-0.6.0-py3-none-any.whl size=259917 sha256=bfb0cd5ca92ce933ef1eb25aa10912082b990e89444ebda92a0eda792f15bbc7
  Stored in directory: /root/.cache/pip/wheels/75/d0/a1/57ea55532e4ff6e3efbec7a851724a8f7a5b073ff648dd4160
  Created wheel for numpy: filename=numpy-1.14.6-cp38-cp38-linux_x86_64.whl size=13509640 sha256=ecd89dfbcbe963497b26337d5ae4a5d0aa1f997a407c5067a3870c23c1de56b2
  Stored in directory: /root/.cache/pip/wheels/77/81/aa/e309a6725c1cb6f5b37c3c67b74828fd4db0827592ff4a4f85
Successfully built gluonnlp numpy
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.1
    Uninstalling numpy-1.19.1:
      Successfully uninstalled numpy-1.19.1
ERROR: After October 2020 you may experience errors when installing or updati

In [7]:
from bert_embedding import BertEmbedding

In [8]:
bert_embedding = BertEmbedding()

Vocab file is not found. Downloading.


In [9]:
doc_body = list(data["body"])

In [10]:
def bert_embed(docs:list,emb_size = 768):
    
    out = []
    for doc in docs:
        sentences = doc.split("\n")
        result = bert_embedding(sentences)
        embeds = np.vstack([np.array(e[1]) for e in result])
        embeds = np.mean(embeds, axis=0) 
        out.append(embeds)
    
    return np.array(out)

In [11]:
out = bert_embed(doc_body)

In [17]:
clustering = AgglomerativeClustering().fit(out)

In [30]:
clusters = AgglomerativeClustering(n_clusters=10,).fit(out)
[i for i in range(len(clusters.labels_)) if clusters.labels_[i]==1]

[1,
 4,
 8,
 10,
 15,
 17,
 20,
 27,
 29,
 38,
 44,
 46,
 47,
 51,
 55,
 57,
 60,
 66,
 67,
 68,
 71,
 76,
 79,
 86,
 88,
 91,
 95]

In [36]:
type(clusters.labels_)

numpy.ndarray

In [38]:
def return_groups(docs:list,clustering:np.ndarray):
    labels = set(clustering)
    
    out = {} 
    for label in labels:
        out[label] = [docs[i] for i in range(len(clustering)) if clustering [i]==label]
    
    return out

In [42]:
return_groups(doc_body,clusters.labels_)[1]

['The friendly skies aren #39;t so friendly to some Delta employees. The airline plans to cut 7,000 jobs in the next 18 months. It #39;s trying to save \\$5 billion by 2006, and that means some major changes.',
 'CHICAGO -- Nothing fazes the Minnesota Twins. Threatened with contraction three years ago, they ran away with the AL Central title.',
 'Description: NPR #39;s Michele Norris talks with Wall Street Journal sports reporter Stefan Fatsis about the latest news from the college gridiron.',
 'SAN FRANCISCO (CBS.MW) -- US Airways Group and its pilots union have resumed negotiations, the Air Line Pilots Association said Wednesday afternoon.',
 'The quest for a wireless, robust video game controller is a hit and miss task, says columnist Daniel Etherington.',
 'The FCC has voted to approve a spectrum swap for Nextel to keep emergency radio spectrum clean from interference. We let you know that it was likely to be approved, as late as yesterday.',
 'November 15, 2004 -- (WEB HOST INDUST